# **Basically set the modules**

In [1]:
# Moundt the G Drive

from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 2.4MB 24.8MB/s 
     |████████████████████████████████| 92kB 5.9MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
%tensorflow_version 1.x
import time
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from konlpy.tag import Okt
import numpy as np
import gensim

TensorFlow 1.x selected.


In [4]:
tf.__version__

'1.15.2'

# **Define the Word2Vec, Bi_LSTM functions**

In [0]:
class Word2Vec():
    
    def __init__(self):
        None

    def tokenize(self, doc):
        pos_tagger = Okt()
        return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]
    
    def read_data(self, filename):
        with open(filename, 'r',encoding='utf-8') as f:
            data = [line.split('\t') for line in f.read().splitlines()]
            data = data[1:]
        return data  
    
    def Word2vec_model(self, model_name):
        
        model = gensim.models.word2vec.Word2Vec.load(model_name)
        return model
    
    def Convert2Vec(self, model_name, doc):  # Convert corpus into vectors
        word_vec = []
        model = gensim.models.word2vec.Word2Vec.load(model_name)
        for sent in doc:
            sub = []
            for word in sent:
                if(word in model.wv.vocab):
                    sub.append(model.wv[word])
                else:
                    sub.append(np.random.uniform(-0.25,0.25,300)) # used for Out Of Vacabulary words
            word_vec.append(sub)
        
        return np.array(word_vec)
    
    def Zero_padding(self, train_batch_X, Batch_size, Maxseq_length, Vector_size):
        
        zero_pad = np.zeros((Batch_size, Maxseq_length, Vector_size))
        for i in range(Batch_size):
            zero_pad[i,:np.shape(train_batch_X[i])[0],:np.shape(train_batch_X[i])[1]] = train_batch_X[i]
            
        return zero_pad
    
    def One_hot(self, data):
       
        index_dict = {value:index for index,value in enumerate(set(data))}
        result = []
        
        for value in data:
            
            one_hot = np.zeros(len(index_dict))
            index = index_dict[value]
            one_hot[index] = 1
            result.append(one_hot)
        
        return np.array(result)

In [0]:
class Bi_LSTM():
    
    def __init__(self, lstm_units, num_class, keep_prob):
        
        self.lstm_units = lstm_units
        
        # Define forward LSTM layer
        with tf.variable_scope('forward', reuse = tf.AUTO_REUSE):
            
            self.lstm_fw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_fw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_fw_cell, output_keep_prob = keep_prob) # 데이터를 droupout 한다
            #드롭아웃은 합성곱 신경망의 오버피팅을 방지하는 가장 유명하면서도 흥미로운 방법이다.
            #드롭아웃 레이어는 뉴런의 일부를 확률적으로 ‘비활성화’한다.
            #이는 뉴런의 상호 적응을 방지하고 피쳐를 개별적으로 학습하도록 강제하여
            #사람이 그림을 맞출때 일부를 손으로 가린채 특징을 학습하여 맞추도록 하는 방식과 유사하게 동작한다.

        # Define backward LSTM layer    
        with tf.variable_scope('backward', reuse = tf.AUTO_REUSE):
            
            self.lstm_bw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_bw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_bw_cell, output_keep_prob = keep_prob)
        
        # Define weight variables
        with tf.variable_scope('Weights', reuse = tf.AUTO_REUSE):
           
            self.W = tf.get_variable(name="W", shape=[2 * lstm_units, num_class],
                                dtype=tf.float32, initializer = tf.contrib.layers.xavier_initializer())
            self.b = tf.get_variable(name="b", shape=[num_class], dtype=tf.float32,
                                initializer=tf.zeros_initializer())
            
            
    def logits(self, X, W, b, seq_len):
        
        # Implement bidirectional_lstm using the function bidirectional_dynamic_rnn.
        (output_fw, output_bw), states = tf.nn.bidirectional_dynamic_rnn(self.lstm_fw_cell, self.lstm_bw_cell,
                                                                         dtype=tf.float32, inputs = X, sequence_length = seq_len)
        # Concat fw, bw final states
        # Extract the last state value
        outputs = tf.concat([states[0][1], states[1][1]], axis=1)

        # set the Linear Regression
        pred = tf.matmul(outputs, W) + b        
        return pred
        
    def model_build(self, logits, labels, learning_rate = 0.001):
        
        with tf.variable_scope("loss"):
            
            # 손실함수로 크로스 엔트로피 오차를 사용한다
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits , labels = labels)) # Softmax loss
            # 최적화는 손실함수를 이용해서 모델을 학습하는 방법으로
            # 손실함수의 결과값을 최소화하는 모델 인자를 찾는다
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss) # Adam Optimizer
            
        return loss, optimizer
    
    def graph_build(self):
        
        self.loss = tf.placeholder(tf.float32)
        self.acc = tf.placeholder(tf.float32)
        
        tf.summary.scalar('Loss', self.loss)
        tf.summary.scalar('Accuracy', self.acc)
        merged = tf.summary.merge_all()
        return merged

손실함수 정의

손실 함수란 신경망이 학습할 때, 정확도에 가까워질 수 있도록 최적화해주는 지표.



머신러닝 모델의 출력값과 사용자가 원하는 출력값의 차이,

즉 오차가 최소화되도록 하는 가중치와 편향을 찾는 것이 바로 학습이다.



![대체 텍스트](https://www.slipp.net/wiki/download/attachments/30771160/image2018-11-7_19-42-48.png?version=1&modificationDate=1541587368000&api=v2)


# **Train the Bi_LSTM model**

In [0]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

W2V = Word2Vec()
train_data = W2V.read_data('/content/gdrive/My Drive/Colab Notebooks/GraduationProject/Word2Vec/Movie_rating_data/ratings_train.txt')

In [8]:
## tokenize the data we have : about 30 mins

print("Tokenize Start!\nCould take minutes...")
train_tokens = [[W2V.tokenize(row[1]),int(row[2])] for row in train_data if W2V.tokenize(row[1]) != []]
train_tokens = np.array(train_tokens)
print("Tokenize Done!")

Tokenize Start!
Could take minutes...
Tokenize Done!


In [0]:
train_X = train_tokens[:,0]
train_Y = train_tokens[:,1]

In [10]:
train_Y_ = W2V.One_hot(train_Y)  ## Convert to One-hot
train_X_ = W2V.Convert2Vec("/content/gdrive/My Drive/Colab Notebooks/GraduationProject/Word2Vec/Word2vec.model",train_X)  ## import word2vec model where you have trained before

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# set the Hyper parameter
Batch_size = 32
Total_size = len(train_X)
Vector_size = 300 # 임베딩 사이즈
train_seq_length = [len(x) for x in train_X]
Maxseq_length = max(train_seq_length) # 여기서는 95
learning_rate = 0.001 # 학습률
lstm_units = 128 # 128차원
num_class = 2 # negative and positive
# num_class = 12 # number of the news categories
training_epochs = 13
keep_prob = 0.75

In [0]:
# 데이터가 들어가기때문에 데이터 갯수가 none으로 지정되고 maxseq_length, vector_size로 이루어진 input shape이다 
X = tf.placeholder(tf.float32, shape = [None, Maxseq_length, Vector_size], name = 'X')
# 데이터 갯수와 데이터가 긍정인지 부정인지를 나타내는 클래스 가 들어간 output shape이다
Y = tf.placeholder(tf.float32, shape = [None, num_class], name = 'Y')
seq_len = tf.placeholder(tf.int32, shape = [None])
keep_prob = tf.placeholder(tf.float32, shape = None) # dropout 할때 keep할 비율

In [13]:
# Bi_LSTM을 불러온다
BiLSTM = Bi_LSTM(lstm_units, num_class, keep_prob)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


In [14]:
# 로짓, 로스, 옵티마이저를 정의해준다
with tf.variable_scope("loss", reuse = tf.AUTO_REUSE):
    logits = BiLSTM.logits(X, BiLSTM.W, BiLSTM.b, seq_len)
    loss, optimizer = BiLSTM.model_build(logits, Y, learning_rate)

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
prediction = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

total_batch = int(len(train_X) / Batch_size)

In [0]:
saver = tf.train.Saver()

train_acc = []
train_loss = []

In [17]:
sess = tf.Session(config = config)
sess.run(init)
train_writer = tf.summary.FileWriter('/content/gdrive/My Drive/Colab Notebooks/GraduationProject/Bi_LSTM', sess.graph)
merged = BiLSTM.graph_build()

# 1 epoch about 40 mins
for epoch in range(training_epochs):

    avg_acc, avg_loss = 0. , 0. # avg_acc, avg_loss initial value set
    mask = np.random.permutation(len(train_X_))
    train_X_ = train_X_[mask]
    train_Y_ = train_Y_[mask]


    for step in range(total_batch):

        train_batch_X = train_X_[step*Batch_size : step*Batch_size+Batch_size]
        train_batch_Y = train_Y_[step*Batch_size : step*Batch_size+Batch_size]
        batch_seq_length = train_seq_length[step*Batch_size : step*Batch_size+Batch_size]
        
        # 각 배치마다 제로 패딩을 해준다
        train_batch_X = W2V.Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size)
        
        # 패딩을 해준 후에 옵티마이즈를 해준다
        sess.run(optimizer, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})

        # Compute average loss
        loss_ = sess.run(loss, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length, keep_prob : 0.75})
        avg_loss += loss_ / total_batch
        
        acc = sess.run(accuracy , feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length, keep_prob : 0.75})
        avg_acc += acc / total_batch
        if step % 100 == 0:
          print("epoch : {:02d} step : {:04d} loss = {:.6f} accuracy= {:.6f}".format(epoch+1, step+1, loss_, acc))

save_path = saver.save(sess,'/content/gdrive/My Drive/Colab Notebooks/GraduationProject/Bi_LSTM/BiLSTM_model_Epoch_13.ckpt')
print ('Save Complete')
print ('save_path : ',save_path)

sess.close()

epoch : 01 step : 0001 loss = 0.597761 accuracy= 0.656250
epoch : 01 step : 0101 loss = 0.559672 accuracy= 0.750000
epoch : 01 step : 0201 loss = 0.458909 accuracy= 0.750000
epoch : 01 step : 0301 loss = 0.522604 accuracy= 0.687500
epoch : 01 step : 0401 loss = 0.446483 accuracy= 0.875000
epoch : 01 step : 0501 loss = 0.368069 accuracy= 0.843750
epoch : 01 step : 0601 loss = 0.417968 accuracy= 0.750000
epoch : 01 step : 0701 loss = 0.409353 accuracy= 0.812500
epoch : 01 step : 0801 loss = 0.480773 accuracy= 0.750000
epoch : 01 step : 0901 loss = 0.588139 accuracy= 0.718750
epoch : 01 step : 1001 loss = 0.522390 accuracy= 0.812500
epoch : 01 step : 1101 loss = 0.461380 accuracy= 0.718750
epoch : 01 step : 1201 loss = 0.538001 accuracy= 0.750000
epoch : 01 step : 1301 loss = 0.472524 accuracy= 0.781250
epoch : 01 step : 1401 loss = 0.495677 accuracy= 0.781250
epoch : 01 step : 1501 loss = 0.333201 accuracy= 0.843750
epoch : 01 step : 1601 loss = 0.482548 accuracy= 0.812500
epoch : 01 ste

In [0]:
# Whole learning time (s)

14409/60

240.15